In [5]:
import cv2
import numpy as np
import os

# Cargar YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers_indices = net.getUnconnectedOutLayers()

output_layers = [layer_names[i - 1] for i in output_layers_indices]


# Obtener la lista de archivos en la carpeta
folder_path = "images"
image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

# Inicializar una lista para almacenar todas las imágenes con objetos detectados
combined_image = None

# Iterar sobre las imágenes y detectar objetos
for image_file in image_files:
    # Cargar imagen
    image_path = os.path.join(folder_path, image_file)
    image = cv2.imread(image_path)
    height, width, channels = image.shape

    # Detectar objetos
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Mostrar información en la pantalla
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Objeto detectado
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Coordenadas del rectángulo
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Eliminar cajas repetidas usando Non-Maximum Suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Dibujar las cajas alrededor de los objetos detectados
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = (0, 255, 0)
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            cv2.putText(image, label, (x, y - 10), font, 1, color, 2)

    # Redimensionar la imagen para que todas tengan la misma altura
    desired_height = 400  # Especifica la altura deseada para todas las imágenes
    ratio = desired_height / height
    image_resized = cv2.resize(image, (int(width * ratio), desired_height))

    # Concatenar la imagen actual con las imágenes anteriores
    if combined_image is None:
        combined_image = image_resized
    else:
        combined_image = np.hstack((combined_image, image_resized))

# Mostrar la imagen resultante con todas las imágenes combinadas
cv2.imshow("Combined Image", combined_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
